# Loopy

 * [GitHub](http://github.com/inducer/loopy)]
 * [Documentation](http://documen.tician.de/loopy)
 * [Homepage](http://mathema.tician.de/software/loopy)
 
 ```sh
 pip3 install loo.py pyopencl
 ```


In [24]:
from pyopencl.tools import get_test_platforms_and_devices

In [25]:
get_test_platforms_and_devices()

[(<pyopencl.Platform 'Apple' at 0x7fff0000>,
  [<pyopencl.Device 'Intel(R) Core(TM) i7-3667U CPU @ 2.00GHz' on 'Apple' at 0xffffffff>,
   <pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>])]

In [26]:
import numpy as np
import pyopencl as cl
import pyopencl.array

In [36]:
%load_ext pyopencl.ipython_ext


The pyopencl.ipython_ext extension is already loaded. To reload it, use:
  %reload_ext pyopencl.ipython_ext


**Context name must be "ctx" or "cl_ctx"**

In [32]:
platform = cl.get_platforms()[0]    # Select the first platform [0]
device = platform.get_devices()[0]  # Select the first device on this platform [0]
cl_ctx = cl.Context([device])       # Create a context with your device
queue = cl.CommandQueue(cl_ctx)     # Create a command queue with your context

In [37]:
n = 10000

a = cl.array.empty(queue, n, dtype=np.float32)
a.fill(15)

b_host = np.random.randn(n).astype(np.float32)
b = cl.array.to_device(queue, b_host)

c = cl.array.empty_like(a)

In [38]:
%%cl_kernel -o "-cl-fast-relaxed-math"

__kernel void sum_vector(__global const float *a,
__global const float *b, __global float *c)
{
  int gid = get_global_id(0);
  c[gid] = a[gid] + b[gid];
}

In [35]:
sum_vector(queue, (n,), None, a.data, b.data, c.data)

c.get()

array([ 13.53202629,  14.00289536,  14.84933758, ...,  15.21384239,
        15.27578449,  16.31548309], dtype=float32)

## Example: double the entries of a vector

In [1]:
import numpy as np
import loopy as lp
import pyopencl as cl
import pyopencl.array

# setup
# -----
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

n = 15 * 10**6
a = cl.array.arange(queue, n, dtype=np.float32)

# create
# ------
knl = lp.make_kernel(
        "{ [i]: 0<=i<n }",
        "out[i] = 2*a[i]")

# transform
# ---------
knl = lp.split_iname(knl, "i", 128, outer_tag="g.0", inner_tag="l.0")

# execute
# -------
evt, (out,) = knl(queue, a=a)

In [2]:
out

array([  0.00000000e+00,   2.00000000e+00,   4.00000000e+00, ...,
         2.99999940e+07,   2.99999960e+07,   2.99999980e+07], dtype=float32)

Components of a loopy kernel:

- The loop domain: `{ [i]: 0<=i<n }`. This tells loopy the values that you would like your loop variables to assume. It is written in [ISL syntax](https://documen.tician.de/loopy/ref_kernel.html#isl-syntax).

- The instructions to be executed. These are generally scalar assignments between array elements, consisting of a left hand side and a right hand side. See Assignment objects for the full syntax of an assignment.